# Using Automated Machine Learning

There are many kinds of machine learning algorithm that you can use to train a model, and sometimes it's not easy to determine the most effective algorithm for your particular data and prediction requirements. Additionally, you can significantly affect the predictive performance of a model by preprocessing the training data, using techniques such as normalization, missing feature imputation, and others. In your quest to find the *best* model for your requirements, you may need to try many combinations of algorithms and preprocessing transformations; which takes a lot of time and compute resources.

Azure Machine Learning enables you to automate the comparison of models trained using different algorithms and preprocessing options. You can use the visual interface in [Azure Machine Learning studio](https://ml/azure.com) or the SDK to leverage this capability. he SDK gives you greater control over the settings for the automated machine learning experiment, but the visual interface is easier to use. In this lab, you'll explore automated machine learning using the SDK.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [4]:
import pip 
for i in pip.get_installed_distributions(local_only=True):
    print(i)

zipp 3.1.0
zict 1.0.0
xlwt 1.2.0
XlsxWriter 0.9.6
xlrd 1.0.0
wrapt 1.11.2
widgetsnbextension 2.0.0
wheel 0.30.0
Werkzeug 0.16.0
websocket-client 0.57.0
wcwidth 0.1.7
vega-datasets 0.7.0
urllib3 1.25.9
unicodecsv 0.14.1
typing-extensions 3.7.4
traitlets 4.3.2
tqdm 4.46.1
tornado 6.0.3
torch 1.3.0
toolz 0.10.0
testpath 0.3
terminado 0.6
termcolor 1.1.0
tensorflow 1.14.0
tensorflow-estimator 1.14.0
tensorboard 1.14.0
tblib 1.4.0
tables 3.3.0
sympy 1.0
statsmodels 0.10.1
SQLAlchemy 1.1.9
spyder 3.1.4
sortedcontainers 2.1.0
sortedcollections 0.5.3
snowballstemmer 1.2.1
smart-open 1.8.4
sklearn-pandas 1.7.0
skl2onnx 1.4.9
six 1.15.0
singledispatch 3.4.0.3
simplegeneric 0.8.1
shap 0.34.0
setuptools 41.4.0
SecretStorage 3.1.2
seaborn 0.9.0
scipy 1.1.0
scikit-learn 0.20.3
scikit-image 0.15.0
s3transfer 0.2.1
ruamel.yaml 0.16.10
ruamel.yaml.clib 0.2.0
rope-py3k 0.9.4.post1
retrying 1.3.3
Resource 0.2.1
requests 2.24.0
requests-oauthlib 1.3.0
QtPy 1.2.1
qtconsole 4.3.0
QtAwesome 0.4.4
pyzmq 16.0.

In [5]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.get(name='dp100-workspace',
 subscription_id='98214768-053e-4f2d-bbd7-5db5549e5c42',
 resource_group='dp100-rg')

print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.8.0 to work with dp100-workspace

## Prepare Data for Automated Machine Learning

You don't need to create a training script for automated machine learning, but you do need to load the training data. In this case, you'll create a dataset containing details of diabetes patients (just as you did in previous labs), and then split this into two datasets: one for training, and another for model validation.

In [6]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# Split the dataset into training and validation subsets
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Dataset already registered.
Data ready!

## Prepare a Compute Target

One of the benefits of cloud compute is that it scales on-demand, enabling you to provision enough compute resources to process multiple runs of an experiment in parallel.

You'll use the Azure Machine Learning compute cluster you created in an earlier lab (if it doesn't exist, it will be created).

> **Important**: Change *your-compute-cluster* to the name of your compute cluster in the code below before running it!

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

try:
    # Get the cluster if it exists
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned

## Configure Automated Machine Learning

Now you're ready to configure the automated machine learning experiment. To do this, you'll need a run configuration that includes the required packages for the experiment environment, and a set of configuration settings that specifies how many combinations to try, which metric to use when evaluating models, and so on.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=4,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## Run an Automated Machine Learning Experiment

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: In this lab, you'll run the automated machine learning experiment with a maximum of 6 iterations to save time. In reality, you might want to try many more iterations using a training cluster.

In [ ]:
from azureml.core.experiment import Experiment
## from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
##RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Determine the Best Performing Model

When the experiment has completed, view the output in the widget, and click the run that produced the best result to see its details.
Then click the link to view the experiment details in the Azure portal and view the overall experiment details before viewing the details for the individual run that produced the best result. There's lots of information here about the performance of the model generated.

Let's get the best run and the model that it produced.

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Automated machine learning includes the option to try preprocessing the data, which is accomplished through the use of [Scikit-Learn transformation pipelines](https://scikit-learn.org/stable/modules/compose.html#combining-estimators) (not to be confused with Azure Machine Learning pipelines!). These produce models that include steps to transform the data before inferencing. You can view the steps in a model like this:

In [ ]:
for step in fitted_model.named_steps:
    print(step)

Finally, having found the best performing model, you can register it.

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **More Information**: For more information Automated Machine Learning, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train).